In [1]:
import plotly.graph_objects as go
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import os
import re

In [6]:

freq = 0
# Initialize an empty DataFrame
df = pd.DataFrame()

# Regular expression pattern for extracting numbers
pattern = re.compile(r'qubits_(\d+)_circ_layers_(\d+)')

# Keywords to look for in the directory names

keywords = ["SimplifiedTwoDesign", "local", "periodic_layers_03"]

# Traverse the logs directory
for root, dirs, files in os.walk("logs"):
    # Check if all keywords are present in the root directory
    if all(keyword in root for keyword in keywords):
        for file in files:
            if file == "abs_coeffs_avg.npy":
                # Extract path components
                path_parts = "/".join(root.split("/")[1:])  # Exclude 'logs/' part

                # Use regex to extract numbers for qubits and circuit layers
                match = pattern.search(path_parts)
                if match:
                    qubits = int(match.group(1))
                    circ_layers = int(match.group(2))

                    # Construct the full file path and read the .npy file
                    file_path = os.path.join(root, file)
                    
                    data = np.load(file_path)

                    # Extract the first element of the array
                    value = data[freq]

                    # Populate the DataFrame
                    if circ_layers not in df.index:
                        df.loc[circ_layers, qubits] = value
                    else:
                        df.at[circ_layers, qubits] = value

# Sort DataFrame by index and columns
df.sort_index(inplace=True)
df.sort_index(axis=1, inplace=True)

# Add index and column titles
df.index.name = 'circuit_layers'
df.columns.name = 'n_qubits'

print(df)


n_qubits               2         3         4
circuit_layers                              
1               0.487734  0.502726  0.499219
2               0.490255  0.496897  0.498622
3               0.499382  0.499622  0.497428
4               0.501306  0.500296  0.497103


In [8]:
z = df.values
x = df.index.values  # circuit_layers
y = df.columns.values.astype(str)  # n_qubits, converted to strings to match your DataFrame



fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

fig.update_layout(
    title=f"c_{freq}",  # Using f-string to dynamically set title
    autosize=False,
    width=1000,
    height=1000,
    margin=dict(l=65, r=50, b=65, t=90),
    scene=dict(
        xaxis_title="circ Layers",  # X-axis label
        yaxis_title="n_qubits",  # Y-axis label
        zaxis_title=f"c_{freq}"  # Z-axis label
    )
)

